In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

自动加载本地文档：

In [2]:
from langchain_chinese.document_loaders import LocalFilesLoader
fl = LocalFilesLoader()
all_docs = fl.load()


In [3]:
fl.documents_folder

'/Users/xuehongwei/github/wencheng/data/documents'

切分文本：

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap = 200,
    length_function = len,
    is_separator_regex = False,
)
chunks = text_splitter.split_documents(all_docs)

文本向量编码：

In [24]:
for chunk in chunks[0:10]:
  print(len(chunk.page_content))

1997
1877
1883
1997
1915
1963
1985
1851
1982
1805


In [25]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(chunks[0:10], OpenAIEmbeddings(model="text-embedding-3-small"))

In [42]:
user_query = "如何管理纳税信用"
result = db.similarity_search(user_query)
result

[Document(page_content='（二）征管模式改革\n\n1.统一数据规范\n\n澳大利亚采用统一数据标准，企业可将数据标准嵌入到其业务、会计软件中；使用统一数据规范，企业提高工作效率，税务局也同样受益于业务部门提交的准确数据，促使数据输入更快速、高效和透明，为后续数据分析提供有力支撑。\n\n2.提升征管体验\n\n电子申报是税收信息化发展中的重要环节，其中一大创新便是支持电子预缴税功能。比利时、丹麦、挪威以及新加坡税务机关在此基础上，开发“默认缴税”功能，即如纳税人逾期申报，税务局则自动征收预缴税款。\n\n3.信用分类分级\n\n澳大利亚、美国、英国等国家的税务机构都基于纳税人的过去行为进行信用评级。针对不同信用程度的纳税人实行不同稽查方式，例如英国税务机构针对信用等级良好的企业实行2到3年稽查一次，而针对信用等级差的企业则每年稽查。\n\n4.智能身份认证\n\n澳大利亚、新加坡等国家的税务机构支持多种方式的身份认证，比如身份证号或个性化账号登陆。除此之外，澳大利亚税务机构还支持声音识别认证功能。多种身份认证方式让更多纳税人使用数字化服务，减低税务局运营成本。\n\n5.智能自动申报\n\n墨西哥税务机构通过对覆盖全国的电子收据、电子发票、电子银行账单等数据进行收集，实现从源头获取数据，减少等待纳税人申报的时间，降低纳税人申报不实数据的风险。\n\n澳大利亚税务机构提供官方软件工具帮助纳税人填写财务报表或申报表，除此之外，基于全国性的统一数据标准，企业也可以通过自动抓取相关数据，自动生成申报表。\n\n现状和问题分析\n\n整体来看，广东省税务局应用系统功能比较完善，渠道覆盖广，基本满足纳税人业务办理需求；数据方面，初步完成了数据资源的统一管理，并制定了相关的制度和规范；IT保障方面，信息化项目管理制度、信息化组织及人才都有基本保障，信息化工作有序推进；基础设施方面，网络、存储、计算等资源较为充足，安全管控到位。同时，在信息化建设过程中也存在一些问题：应用系统架构不统一，系统孤立，缺乏有效协同；业务功能未按照纳税人视角进行设计，用户体验较差；注重系统的开发，但运维投入较低等；数据管理规范需进一步提升和完善；数据系统标准不一、重复建设；数据资源缺乏有效治理与融合等；基础设施资源孤立、缺乏统筹调度、缺少统一技术规范等；组织管理缺少对信息化建设的顶

In [43]:
db.as_retriever()

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x12590f2b0>)

In [44]:
from sentence_transformers import CrossEncoder

model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', max_length=512)

scores = model.predict([(user_query, r.page_content) for r in result])
# 按得分排序
sorted_list = sorted(
    zip(scores, result), key=lambda x: x[0], reverse=True)

sorted_list

[(6.017462,
  Document(page_content='第四章\t整体架构\t38\n\n4.1 总体框架\t38\n\n4.2 业务架构\t39\n\n4.3 应用架构\t41\n\n4.4 数据架构\t44\n\n4.5 与其他系统的关系\t46\n\n第五章\t八大创新\t51\n\n5.1 智慧可视的决策指挥中心\t51\n\n5.2 交互协同的税务生态\t53\n\n5.3 移动互联的掌上税务应用\t56\n\n5.4 精准画像的信用风险管税\t57\n\n5.5 统一集成的数据资源中心\t61\n\n5.6 安全可控的税务应用\t63\n\n5.7 全程智能的自主运维\t65\n\n5.8 融合集智的联创中心\t67\n\n第六章\t三大应用\t70\n\n6.1 征管应用\t70\n\n6.2 数据应用\t80\n\n6.3 行管应用\t86\n\n第七章\t六大平台\t90\n\n7.1 渠道管理平台\t90\n\n7.2 统一工作平台\t92\n\n7.3 通用业务平台\t94\n\n7.4 数据服务平台\t97\n\n7.5 数据管控平台\t100\n\n7.6 运维管理平台\t106\n\n第八章\t基础设施\t111\n\n8.1 “一片云”数据中心建设\t111\n\n8.2 “一张网”云网协同建设\t116\n\n8.3 容灾双活保障\t120\n\n第九章\t安全体系\t123\n\n9.1 安全管理体系\t123\n\n9.2 安全技术体系\t124\n\n9.3 安全运维体系\t126\n\n9.4 数据安全体系\t128\n\n第十章\t保障体系\t131\n\n10.1 组织保障\t131\n\n10.2 人才保障\t137\n\n10.3 制度保障\t141\n\n\t\t第十一章\t项目投资\t146\n\n11.1 规划项目\t146\n\n\t11.1.1 总体规划项目\t146\n\n\t11.1.2 政务服务项目\t148\n\n\t11.1.3 基础设施项目\t151\n\n\t11.1.4 数据管理项目\t154\n\n\t11.1.5 数据呈现项目\t158\n\n11.2 投资匡算\t162\n\n\t\t第十二章\t实施策略\t164\n\n12.1 实施策略\t164\n\n12.2 实